In [ ]:
import numpy as np
import pandas as pd
import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
from echopop import Survey
from echopop.spatial.variogram import get_variogram_arguments
import copy
from echopop.spatial.transect import edit_transect_columns
from echopop.spatial.projection import transform_geometry
from echopop.analysis import variogram_analysis
from echopop.spatial.mesh import griddify_lag_distances
from echopop.spatial.variogram import empirical_variogram
from IPython.display import display, clear_output
from echopop.spatial.variogram import variogram

In [ ]:

survey = Survey(init_config_path="C:/Users/Brandyn/Documents/GitHub/echopop/config_files/initialization_config.yml" ,
                survey_year_config_path="C:/Users/Brandyn/Documents/GitHub/echopop/config_files/survey_year_2019_config.yml")
survey.transect_analysis()
self = survey
settings_dict = {"variable": "biomass_density",
                 "stratum_name": "stratum_num",
                 "kriging_parameters": {
                    "longitude_reference": -124.78338,
                    "latitude_offset": 45.0,
                    "longitude_offset": -124.78338,
                 }}
# Prepare the transect data
# ---- Create a copy of the transect dictionary
transect_input = copy.deepcopy(self.analysis["transect"])
# ---- Edit the transect data
transect_data = edit_transect_columns(transect_input, settings_dict)
isobath_df = self.input["statistics"]["kriging"]["isobath_200m_df"]
transect_data, _, _ = transform_geometry(transect_data, isobath_df, settings_dict)

# Get the stratum name
stratum_name = self.analysis["settings"]["transect"]["stratum_name"]

# Get standardization config for kriging 
standardization_parameters = self.input["statistics"]["kriging"]["model_config"]
# ---- Get isobath data
isobath_df = self.input["statistics"]["kriging"]["isobath_200m_df"]

# Get variogram parameters
variogram_parameters = self.input["statistics"]["variogram"]["model_config"].copy()

# Generate settings dictionary
settings_dict = {
    "stratum_name": stratum_name,
    "verbose": False,
    "kriging_parameters": {
        "longitude_reference": standardization_parameters["longitude_reference"],
        "longitude_offset": standardization_parameters["longitude_offset"],
        "latitude_offset": standardization_parameters["latitude_offset"],
    },
}


# # Create a copy of the existing variogram settings
# variogram_parameters = self.input["statistics"]["variogram"]["model_config"].copy()
# n_lags = 30
# lag_resolution = variogram_parameters["lag_resolution"]
# # ---- Number of lags
# variogram_parameters["n_lags"] = 30
# # ---- Azimuth range
# variogram_parameters["azimuth_range"] = 360
# # ---- Force lag-0
# variogram_parameters["force_lag_zero"] = True
# # Compute the lag distances
# distance_lags = np.arange(1, n_lags) * lag_resolution
# # ---- Add to the `variogram_parameters` dictionary
# variogram_parameters["distance_lags"] = distance_lags
# # ---- Update the max range parameter, if necessary
# max_range = lag_resolution * n_lags
# lags, gamma_h, lag_counts, lag_covariance = empirical_variogram(
#     transect_data, variogram_parameters, {"variable": "biomass_density"}
# )

In [ ]:
VARIOGRAM_MODEL_PARAMETER_MAP = {
    "Bessel-exponential": ["bessel", "exponential"],
    "Bessel": "bessel",
    "Exponential": "exponential",
    "Gaussian": "gaussian",
    "Linear": "linear",
    "Sinc": "sinc",
    "Spherical": "spherical",
    "Bessel-Gaussian": ["bessel", "gaussian"],
    "Cosine-exponential": ["cosine", "exponential"],
    "Cosine-Gaussian": ["cosine", "gaussian"],
    "Exponential-linear": ["exponential", "linear"],
    "Gaussian-linear": ["gaussian", "linear"]
}

# Variogram model parameter map
THEORETICAL_VARIOGRAM_PARAMETER_MAP = {
    "nugget": dict(name="Nugget", widget="entry", step=0.01),
    "sill": dict(name="Sill", widget="entry", step=0.01),
    "correlation_range": dict(name="Correlation range", widget="entry", step=0.0001),
    "decay_power": dict(name="Decay power", widget="entry", step=0.05),
    "hole_effect_range": dict(name="Hole effect range", widget="entry", step=0.0001),
    "enhance_semivariance": dict(name="Enhance semivariance", widget="checkbox", step=np.nan)
}

# Empirical variogram general settings map
EMPIRICAL_VARIOGRAM_PARAMETER_MAP = {
    "n_lags": 30,
    "lag_resolution": 0.002,
    "azimuth_range": 360.0
}

# Optimization
OPTIMIZATION_PARAMETER_MAP = {
    "Maximum function evaluations": dict(name="max_fun_evaluations", widget="entry_int", value=500, step=1),
    "Cost function tolerance": dict(name="cost_fun_tolerance", widget="entry_float", value=1e-6, step=1e-7),
    "Solution tolerance": dict(name="solution_tolerance", widget="entry_float", value=1e-6, step=1e-5),
    "Gradient tolerance": dict(name="gradient_tolerance", widget="entry_float", value=1e-4, step=1e-5),
    "Finite differences step size": dict(name="finite_step_size", widget="entry_float", value=1e-8, step=1e-9),
    "Trust Region solver method": dict(name="trust_region_solver", widget="entry_str", value="exact", step=np.nan),
    "Characteristic x-scale": dict(name="x_scale", widget="entry_hybrid", value="jacobian", step=np.nan),
    "Jacobian approximation method": dict(name="jacobian_approx", widget="entry_str", value="central", step=np.nan)
}

style = {'description_width': '150px'}
layout = {'width': '400px'}

In [ ]:
empirical_variogram_dropdown = widgets.Dropdown(
    options=list(["Biomass (biomass density)", "Abundance (number density)"]),
    value="Biomass (biomass density)",
    description="Transect variable",
    disabled=False,
    style = {"description_width": "150px"}, 
    layout = {"width": "400px"}
)   

# Create general settings value entry widgets
# ---- Number of lags
n_lags_entry = widgets.IntText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["n_lags"],
                                description="Number of lags",
                                step=1, style=style, layout=layout)
# ---- Lag (increment) resolution
lag_resolution_entry =  widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"], 
                                            description='Lag resolution', 
                                            step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"]/2, 
                                            style=style, layout=layout)
# ---- Azimuth range 
azimuth_range_entry = widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"], 
                                        description='Azimuth Range', 
                                        step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"]/360, 
                                        style=style, layout=layout)

# Initialize the value dictionary
general_settings = {
    "n_lags": n_lags_entry.value,
    "lag_resolution": lag_resolution_entry.value,
    "azimuth_range": azimuth_range_entry.value,
    "variable": empirical_variogram_dropdown.value,
}

settings_dict.update({"variable": "biomass_density"})

def update_settings_dict(change):
    settings_dict.update({
        "variable": (
            "biomass_density" if empirical_variogram_dropdown.value == "Biomass (biomass density)" 
            else "number_density"
        ),
    })

empirical_variogram_dropdown.observe(update_settings_dict, names='value')

widgets.VBox([n_lags_entry, lag_resolution_entry, azimuth_range_entry, empirical_variogram_dropdown])



In [ ]:
print(settings_dict)

In [ ]:
from functools import partial

def stitch_variogram_accordion(settings_widget, variogram_widget, optimization_widget):

    # Initialize the accordion tabs
    variogram_accordion = widgets.Accordion(layout=dict(width="450px"))

    # Populate the accordion children
    variogram_accordion.children = [settings_widget, variogram_widget, optimization_widget]

    # Set the accordion tab titles
    variogram_accordion.set_title(0, "Empirical variogram", )
    variogram_accordion.set_title(1, "Theoretical variogram", )
    variogram_accordion.set_title(2, "Optimize variogram parameters", )

    # Return the accordion
    return variogram_accordion

def get_variogram_defaults(variogram_parameters, argument):
    DEFAULT_PARAMETERS = {
        "nugget": 0.0,
        "sill": 1.0,
        "hole_effect_range": 0.0,
        "decay_power": 1.5,
        "enhance_semivariance": False,
    }
    # ---- get variogram config
    for argument in DEFAULT_PARAMETERS.keys():
        if argument in variogram_parameters.keys():
            return variogram_parameters[argument]
        else:
            return DEFAULT_PARAMETERS[argument]
        
def theoretical_variogram_widget(empirical_params):
    # Define your dropdown widget
    dropdown_variogram_model = widgets.Dropdown(
        options=list(VARIOGRAM_MODEL_PARAMETER_MAP.keys()),
        value="Bessel-exponential",
        description="Variogram model",
        disabled=False,
        style = {"description_width": "153px"}, 
        layout = {"width": "403px"}
    )

    # Create text entry widgets (initially empty)
    text_widgets = {
        param_name: widgets.FloatText(description=param_details['name'], step=param_details['step'])
        if param_details['widget'] == 'entry' else
        widgets.Checkbox(description=param_details['name'])
        for param_name, param_details in THEORETICAL_VARIOGRAM_PARAMETER_MAP.items()
    }

    # Create theoretical model computation button
    compute_theoretical_variogram_button = widgets.Button(
        description='Compute Theoretical Variogram',
        layout={"width": "405px"},
        style={'font_weight': 'bold'}
    )

    def get_variogram_defaults(variogram_parameters, argument):
        DEFAULT_PARAMETERS = {
            "nugget": 0.0,
            "sill": 1.0,
            "hole_effect_range": 0.0,
            "decay_power": 1.5,
            "enhance_semivariance": False,
        }
        # ---- get variogram config
        if argument in variogram_parameters.keys():
            return variogram_parameters[argument]
        else:
            return DEFAULT_PARAMETERS[argument]

    # Dynamically populate the model parameter field(s)
    # ---- Initialize the value entry dictionary
    variogram_settings = {}
    # ---- Initialize the Output widget
    variogram_model_widgets = widgets.Output()

    def dynamic_parameters(change):
        with variogram_model_widgets:
            # ---- Clear previous values
            clear_output(wait=True)
            # ---- Get the dropdown selection
            model_selection = change["new"]
            # ---- Retrieve the model argument used for `variogram(...)`
            model_name = VARIOGRAM_MODEL_PARAMETER_MAP[model_selection]
            # ---- Get argument names for this particular function
            function_arguments, _ = get_variogram_arguments(model_name)
            # ---- Model arguments (convert to dictionary)
            args_dict = dict(function_arguments)
            # ---- Clear `variogram_settings`
            variogram_settings.clear()
            # ---- Clear existing widgets, if any
            variogram_model_widgets.clear_output()
            # ---- Add the model name to the dictionary
            variogram_settings["model"] = model_name
            # ---- Generate widgets for each parameter
            for param in args_dict.values():
                # ---- Get the parameter name
                parameter = param.name
                # ---- Search reference dictionary for widget parameterization
                if parameter != "distance_lags":
                    # ---- Get default parameterization
                    default_value = get_variogram_defaults(variogram_parameters, parameter)
                    # ---- Get the function argument name
                    arg_name = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["name"]
                    # ---- Get widget-type
                    widget_type = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["widget"]
                    # ---- Get step size
                    step_size = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["step"]
                    # ---- Create the appropriate widget
                    if widget_type == "checkbox":
                        checkbox = widgets.Checkbox(value=default_value, description=arg_name, 
                                                    style={"description_width": "initial"})
                        # ---- Add checkbox entry to `variogram_settings`
                        variogram_settings[parameter] = checkbox
                        # ---- Display the result
                        display(checkbox)
                    elif widget_type == "entry": 
                        entry = widgets.FloatText(value=default_value, description=arg_name, 
                                                style=style, layout=layout, step=step_size)
                        # ---- Add floattext entry to `variogram_settings`
                        variogram_settings[parameter] = entry
                        # ---- Display the result
                        display(entry)            

    # Attach the observer function to the dropdown menu
    dropdown_variogram_model.observe(dynamic_parameters, names="value")

    # Register the updated parameters
    dynamic_parameters({"new": dropdown_variogram_model.value})

    compute_theoretical_variogram_button.on_click(on_compute_button_clicked)

    widget_box = widgets.VBox([
        dropdown_variogram_model,
        variogram_model_widgets,
        compute_theoretical_variogram_button
    ])

    return widget_box

def compute_theoretical_variogram(parameters, lags):
    # Get the current input values
    arg_dict = {
        key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
        for key, value in parameters.items()
    }

    # pass into variogram
    
    distance_lags = lags
    return variogram(distance_lags, arg_dict)

def plot_theoretical_variogram(fig, ax, empirical_params, parameters):
    lags = empirical_params["lags"]
    theoretical_variogram = compute_theoretical_variogram(parameters, lags)
    
    ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
    # plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()

def plot_empirical_variogram(lags: np.ndarray, gamma_h: np.ndarray, lag_counts: np.ndarray):
    # Generate point-size scaling lambda function
    size_scale = lambda x: (((x - x.min()) / float(x.max() - x.min()) + 1) * 10)**2

    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 5)) # Figure and axis initialization
    scatter = ax.scatter(lags, gamma_h, s=size_scale(lag_counts), edgecolors="black") # scatter plot
    ax.set_xlabel("Lag distance (h)") # x-axis label
    ax.set_ylabel("Semivariance (γ)") # y-axis label
    ax.grid(True) # create axis grid lines
    ax.set_axisbelow(True) # set axis grid lines below points
    plt.tight_layout() # adjust padding around plotting area

    # Generate list of annotation points
    annotation_points = ax.annotate("", xy=(0, 0), xytext=(-50, -50), textcoords="offset points",
                                    bbox=dict(boxstyle="round", fc="white", ec="black", alpha=1.0),
                                    arrowprops=dict(arrowstyle="->", color="black", lw=1, 
                                                    linestyle="-"))
    # ---- Set values to be invisible 
    annotation_points.set_visible(False)

    # Helper function for revealing invisible annotations
    def reveal_annotation(ind):
        position = scatter.get_offsets()[ind["ind"][0]] # get scatter point positions
        annotation_points.xy = position # set the annotation coordinates
        # ---- Create text label
        text = f"h={position[0]:.2f}\nγ={position[1]:.2f}\nCount={lag_counts[ind['ind'][0]]}" 
        annotation_points.set_text(text) # assign text to annotation point
        annotation_points.get_bbox_patch().set_alpha(1.0) # set alpha transparency for text box

    # Helper function for target annotation
    def on_hover(event):
        vis = annotation_points.get_visible() # detect the visible annotation point
        if event.inaxes == ax:
            cont, ind = scatter.contains(event)
            # ---- If the annotation intersects with the cursor hover point
            if cont: 
                reveal_annotation(ind)
                annotation_points.set_visible(True)          
                # ---- Draw
                fig.canvas.draw_idle()
            else:
                if vis: # points that aren't 'highlighted' by the cursor
                    annotation_points.set_visible(False)
                    # ---- Draw
                    fig.canvas.draw_idle()

    # Create event connection
    fig.canvas.mpl_connect("motion_notify_event", on_hover)

    # Clean up the canvas
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

    # Return figure and axis
    return fig, ax

# General settings widget
def empirical_variogram_widget():

    # Create general settings value entry widgets
    # ---- Number of lags
    n_lags_entry = widgets.IntText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["n_lags"],
                                   description="Number of lags",
                                   step=1, style=style, layout=layout)
    # ---- Lag (increment) resolution
    lag_resolution_entry =  widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"], 
                                              description='Lag resolution', 
                                              step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"]/2, 
                                              style=style, layout=layout)
    # ---- Azimuth range 
    azimuth_range_entry = widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"], 
                                            description='Azimuth Range', 
                                            step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"]/360, 
                                            style=style, layout=layout)
    
    # Add variable dropdown widget
    empirical_variogram_dropdown = widgets.Dropdown(
        options=list(["Biomass (biomass density)", "Abundance (number density)"]),
        value="Biomass (biomass density)",
        description="Transect variable",
        disabled=False,
        style = {"description_width": "150px"}, 
        layout = {"width": "400px"}
    )  

    # Add empirical variogram button
    compute_empirical_variogram_button = widgets.Button(description="Compute Empirical Variogram",
                                                        layout={"width": "405px"},
                                                        style={"font_weight": "bold"})    
    
    settings_dict.update({"variable": "biomass_density"})

    def update_settings_dict(change):
        settings_dict.update({
            "variable": (
                "biomass_density" if empirical_variogram_dropdown.value == "Biomass (biomass density)" 
                else "number_density"
            ),
        })

    # Initialize the value dictionary
    general_settings = {
        "n_lags": n_lags_entry.value,
        "lag_resolution": lag_resolution_entry.value,
        "azimuth_range": azimuth_range_entry.value,
        "variable": empirical_variogram_dropdown.value,
    }

    def compute_empirical_variogram(transect_data, general_settings, settings_dict):
        # Attach the update function to the dropdown value change event
        empirical_variogram_dropdown.observe(update_settings_dict, names='value')
        # Compute the distance lags
        distance_lags = np.arange(1, general_settings["n_lags"]) * general_settings["lag_resolution"]
        # ---- Compute the maximum range
        max_range = general_settings["n_lags"] * general_settings["lag_resolution"]

        # Update the general settings for computing the empirical variogram
        general_settings.update({"force_lag_zero": True, "distance_lags": distance_lags, 
                                "range": max_range})
        
        # Compute the empirical variogram
        lags, gamma_h, lag_counts, _ = empirical_variogram(
            transect_data, general_settings, settings_dict
        )

        # Return the outputs
        return lags, gamma_h, lag_counts, general_settings
    
    empirical_params = {}
    # # Button-click
    def click_empirical_variogram_button(button, empirical_params):
        # Show the loading indicator
        loading_label.value = "Computing the empirical variogram, please wait..."
        # Compute the empirical variogram
        
        lags, gamma_h, lag_counts, _ = compute_empirical_variogram(transect_data, general_settings, settings_dict)
        # empirical_params = {}
        empirical_params["lags"] = lags
        empirical_params["gamma_h"] = gamma_h
        empirical_params["lag_counts"] = lag_counts
        # Clear the previous plot
        plot_output_widget.clear_output()

        # Plot the new empirical variogram
        with plot_output_widget:
            fig, ax = plot_empirical_variogram(lags, gamma_h, lag_counts)
            plt.show(fig)

        # Hide the loading indicator
        loading_label.value = ""

        # return empirical_params

    # Attach the button click event to the handler function
    compute_empirical_variogram_button.on_click(lambda button: click_empirical_variogram_button(button, empirical_params))


    # compute_empirical_variogram_button.on_click(click_empirical_variogram_button)

    # Create helper function that updates the current entry values
    def update_general_settings(change):
        # ---- Number of lags
        general_settings["n_lags"] = n_lags_entry.value
        # ---- Lag (increment) resolution
        general_settings["lag_resolution"] = lag_resolution_entry.value
        # ---- Azimuth range 
        general_settings["azimuth_range"] = azimuth_range_entry.value

    def register_observers(widgets, handler, names='value'):
        for widget in widgets:
            widget.observe(handler, names=names)

    # Register the widget observers
    register_observers([n_lags_entry, lag_resolution_entry, azimuth_range_entry], 
                       update_general_settings)
    
    # Vertically concatenate/bundle widgets
    widget_box = widgets.VBox(
        [n_lags_entry, lag_resolution_entry, azimuth_range_entry, empirical_variogram_dropdown,
        compute_empirical_variogram_button]
    )

    # Return the widget and settings dictionary
    return widget_box, general_settings, empirical_params

def plot_theoretical_variogram(fig, ax, empirical_params, parameters):
    lags = empirical_params["lags"]
    theoretical_variogram = compute_theoretical_variogram(parameters, lags)
    
    ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
    # plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()

def on_compute_button_clicked(button):
    # parameters = variogram_widgets
    parameters = variogram_settings
    with plot_output_widget: 
        # out.clear_output(wait=True)  # Clear previous plot
        clear_output(wait=True)
        fig, ax = plot_empirical_variogram(**empirical_params)
        plot_theoretical_variogram(fig, ax, empirical_params, parameters)
        fig.canvas.toolbar_visible = False
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        fig.canvas.resizable = False
        plt.show(fig)
        # plt.close(fig)

In [ ]:
%matplotlib widget
# empirical_params = {}
loading_label = widgets.Label(value="")
plot_output_widget = widgets.Output(layout=widgets.Layout(height="550px"))

variogram_accordion = widgets.Accordion(layout=dict(width="550px"))
tab_empirical_variogram = widgets.Label(value="Optimization Parameters tab will go here.")
tab_contents_variogram = widgets.Label(value="Optimization Parameters tab will go here.")
placeholder_optimization = widgets.Label(value="Optimization Parameters tab will go here.")
tab_contents_optimization = widgets.VBox([placeholder_optimization])

tab_empirical_variogram, general_settings, empirical_params = empirical_variogram_widget()
full_accordion = stitch_variogram_accordion(tab_empirical_variogram, 
                                            theoretical_variogram_widget(empirical_params), 
                                            tab_contents_optimization)

widgets.HBox([full_accordion, plot_output_widget])


In [578]:
OPTIMIZATION_PARAMETER_MAP = {
    "Maximum function evaluations": dict(name="max_fun_evaluations", widget="entry_int", value=500, step=1),
    "Cost function tolerance": dict(name="cost_fun_tolerance", widget="entry_float", value=1e-6, step=1e-7),
    "Solution tolerance": dict(name="solution_tolerance", widget="entry_float", value=1e-6, step=1e-5),
    "Gradient tolerance": dict(name="gradient_tolerance", widget="entry_float", value=1e-4, step=1e-5),
    "Finite differences step size": dict(name="finite_step_size", widget="entry_float", value=1e-8, step=1e-9),
    "Trust Region solver method": dict(name="trust_region_solver", widget="entry_str", value="exact", step=["exact", "base"]),
    "Characteristic x-scale": dict(name="x_scale", widget="entry_hybrid", value="jacobian", value_float=1.0, step=0.1),
    "Jacobian approximation method": dict(name="jacobian_approx", widget="entry_str", value="central", step=["central", "forward"])
}

# Initialize dictionary for tracking argument values
optimization_args = {}
float_widgets = {}
checkbox_widgets = {}

# Create a container for the widgets
widget_entries = []

# Define layout and style
opt_layout = {"width": "420px"}
opt_style = {"description_width": "200px"}

# # Function to update values dictionary
def update_values(change):
    widget = change['owner']
    description = widget.description
    for desc, config in OPTIMIZATION_PARAMETER_MAP.items():
        if desc == description:
            widget_type = config["widget"]
            name = config["name"]

            if widget_type == "entry_int" or widget_type == "entry_float":
                optimization_args[name] = widget.value
            elif widget_type == "entry_str":
                optimization_args[name] = widget.value
            elif widget_type == "entry_hybrid":
                checkbox_value = checkbox_widgets[description].value
                if checkbox_value:
                    optimization_args[name] = "jacobian"
                else:
                    optimization_args[name] = float_widgets[description].value

# Function to update values dictionary
# def update_values(name):
#     def update(change):
#         optimization_args[name] = change.new
#     return update

# Create widgets based on OPTIMIZATION_PARAMETER_MAP
for description, config in OPTIMIZATION_PARAMETER_MAP.items():
    widget_type = config["widget"]
    initial_value = config["value"]
    step = config["step"]
    name = config["name"]

    if widget_type == "entry_int":
        widget = widgets.IntText(value=initial_value, description=description, step=step, 
                                    layout=opt_layout, style=opt_style)
    elif widget_type == "entry_float":
        widget = widgets.FloatText(value=initial_value, description=description, step=step, 
                                    layout=opt_layout, style=opt_style)
    elif widget_type == "entry_str":
        widget = widgets.Dropdown(options=step, value=initial_value, description=description, 
                                    layout=opt_layout, style=opt_style)
    elif widget_type == "entry_hybrid":
        float_widget = widgets.FloatText(value=config["value_float"], description=description, layout=opt_layout, 
                                            style=opt_style, step=step, disabled=True)
        checkbox_widget = (
            widgets.Checkbox(description="Use Jacobian for characteristic x-scaling", 
                                value=True, 
                                layout={"width": "600px"}, 
                                style={"description_width": "170px"})
        )

        # Function to toggle between float widget and disabled state
        def toggle_float_widget(change):
            if change.new:
                optimization_args["x_scale"] = "jacobian"
            else:
                float_widget.disabled = False
                optimization_args["x_scale"] = float(float_widget.value) 
        
        checkbox_widget.observe(toggle_float_widget, 'value')
        
        # Initial state based on default value
        # if initial_value == "jacobian":
        #     float_widget.disabled = True
        
        widget = widgets.VBox([checkbox_widget, float_widget])

    def update_optimization_args(change, name=name):
        optimization_args[name] = change.new
    # Attach observer to each widget to update values dictionary
    widget.observe(update_values, names='value')
    # widget.observe(update_optimization_args, names='value')
    # checkbox_widget.observe(toggle_float_widget, 'value')
    optimization_args[config["name"]] = initial_value 
    # if config["name"] == "x_scale":
    #     if widget.children[0].value:
    #         optimization_args[config["name"]] = "jacobian"
    #     else:
    #         optimization_args[config["name"]] = widget.children[1].value
    # else:
    #     optimization_args[config["name"]] = initial_value    

    # Add the created widget to the list
    widget_entries.append(widget)

# Display the widgets
widgets.VBox(widget_entries)

In [584]:
optimization_args

{'max_fun_evaluations': 500,
 'cost_fun_tolerance': 1e-06,
 'solution_tolerance': 1e-06,
 'gradient_tolerance': 0.0001,
 'finite_step_size': 1.6e-08,
 'trust_region_solver': 'base',
 'x_scale': 1.0,
 'jacobian_approx': 'central'}

In [507]:
widget = widget_entries[6]
widget.children[0].value

False

In [490]:
float_widget.value

1.5

In [ ]:
def empirical_variogram_widget():

    # Create general settings value entry widgets
    # ---- Number of lags
    n_lags_entry = widgets.IntText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["n_lags"],
                                   description="Number of lags",
                                   step=1, style=style, layout=layout)
    # ---- Lag (increment) resolution
    lag_resolution_entry =  widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"], 
                                              description='Lag resolution', 
                                              step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"]/2, 
                                              style=style, layout=layout)
    # ---- Azimuth range 
    azimuth_range_entry = widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"], 
                                            description='Azimuth Range', 
                                            step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"]/360, 
                                            style=style, layout=layout)
    
    # Add variable dropdown widget
    empirical_variogram_dropdown = widgets.Dropdown(
        options=list(["Biomass (biomass density)", "Abundance (number density)"]),
        value="Biomass (biomass density)",
        description="Transect variable",
        disabled=False,
        style = {"description_width": "150px"}, 
        layout = {"width": "400px"}
    )  

    # Add empirical variogram button
    compute_empirical_variogram_button = widgets.Button(description="Compute Empirical Variogram",
                                                        layout={"width": "405px"},
                                                        style={"font_weight": "bold"})    
    
    settings_dict.update({"variable": "biomass_density"})

    def update_settings_dict(change):
        settings_dict.update({
            "variable": (
                "biomass_density" if empirical_variogram_dropdown.value == "Biomass (biomass density)" 
                else "number_density"
            ),
        })

    # Initialize the value dictionary
    general_settings = {
        "n_lags": n_lags_entry.value,
        "lag_resolution": lag_resolution_entry.value,
        "azimuth_range": azimuth_range_entry.value,
        "variable": empirical_variogram_dropdown.value,
    }

    def compute_empirical_variogram(transect_data, general_settings, settings_dict):
        # Attach the update function to the dropdown value change event
        empirical_variogram_dropdown.observe(update_settings_dict, names='value')
        # Compute the distance lags
        distance_lags = np.arange(1, general_settings["n_lags"]) * general_settings["lag_resolution"]
        # ---- Compute the maximum range
        max_range = general_settings["n_lags"] * general_settings["lag_resolution"]

        # Update the general settings for computing the empirical variogram
        general_settings.update({"force_lag_zero": True, "distance_lags": distance_lags, 
                                "range": max_range})
        
        # Compute the empirical variogram
        lags, gamma_h, lag_counts, _ = empirical_variogram(
            transect_data, general_settings, settings_dict
        )

        # Return the outputs
        return lags, gamma_h, lag_counts, general_settings
    
    empirical_params = {}
    # # Button-click
    def click_empirical_variogram_button(button, empirical_params):
        # Show the loading indicator
        loading_label.value = "Computing the empirical variogram, please wait..."
        # Compute the empirical variogram
        
        lags, gamma_h, lag_counts, _ = compute_empirical_variogram(transect_data, general_settings, settings_dict)
        # empirical_params = {}
        empirical_params["lags"] = lags
        empirical_params["gamma_h"] = gamma_h
        empirical_params["lag_counts"] = lag_counts
        # Clear the previous plot
        plot_output_widget.clear_output()

        # Plot the new empirical variogram
        with plot_output_widget:
            fig, ax = plot_empirical_variogram(lags, gamma_h, lag_counts)
            plt.show(fig)

        # Hide the loading indicator
        loading_label.value = ""

        # return empirical_params

    # Attach the button click event to the handler function
    compute_empirical_variogram_button.on_click(lambda button: click_empirical_variogram_button(button, empirical_params))


    # compute_empirical_variogram_button.on_click(click_empirical_variogram_button)

    # Create helper function that updates the current entry values
    def update_general_settings(change):
        # ---- Number of lags
        general_settings["n_lags"] = n_lags_entry.value
        # ---- Lag (increment) resolution
        general_settings["lag_resolution"] = lag_resolution_entry.value
        # ---- Azimuth range 
        general_settings["azimuth_range"] = azimuth_range_entry.value

    def register_observers(widgets, handler, names='value'):
        for widget in widgets:
            widget.observe(handler, names=names)

    # Register the widget observers
    register_observers([n_lags_entry, lag_resolution_entry, azimuth_range_entry], 
                       update_general_settings)
    
    # Vertically concatenate/bundle widgets
    widget_box = widgets.VBox(
        [n_lags_entry, lag_resolution_entry, azimuth_range_entry, empirical_variogram_dropdown,
        compute_empirical_variogram_button]
    )

    # Return the widget and settings dictionary
    return widget_box, general_settings, empirical_params


def stitch_variogram_accordion(settings_widget, variogram_widget, optimization_widget):

    # Initialize the accordion tabs
    variogram_accordion = widgets.Accordion(layout=dict(width="450px"))

    # Populate the accordion children
    variogram_accordion.children = [settings_widget, variogram_widget, optimization_widget]

    # Default first tab
    variogram_accordion.selected_index = 0

    # Set the accordion tab titles
    variogram_accordion.set_title(0, "Empirical variogram", )
    variogram_accordion.set_title(1, "Theoretical variogram", )
    variogram_accordion.set_title(2, "Optimize variogram parameters", )

    # Return the accordion
    return variogram_accordion

def get_variogram_defaults(variogram_parameters, argument):
    DEFAULT_PARAMETERS = {
        "nugget": 0.0,
        "sill": 1.0,
        "hole_effect_range": 0.0,
        "decay_power": 1.5,
        "enhance_semivariance": False,
    }
    # ---- get variogram config
    if argument in variogram_parameters.keys():
        return variogram_parameters[argument]
    else:
        return DEFAULT_PARAMETERS[argument]


def theoretical_variogram_widgets(empirical_params):
    dropdown_variogram_model = widgets.Dropdown(
        options=list(VARIOGRAM_MODEL_PARAMETER_MAP.keys()),
        value="Bessel-exponential",
        description="Variogram model",
        disabled=False,
        style = {"description_width": "153px"}, 
        layout = {"width": "403px"}
    )

    # Create theoretical model computation button
    compute_theoretical_variogram_button = widgets.Button(
        description='Compute Theoretical Variogram',
        layout={"width": "405px"},
        style={'font_weight': 'bold'}
    )

    # Dynamically populate the model parameter field(s)
    # ---- Initialize the value entry dictionary
    variogram_settings = {}
    # ---- Initialize the Output widget
    variogram_model_widgets = widgets.Output()

    def dynamic_parameters(change):
        with variogram_model_widgets:
            # ---- Clear previous values
            clear_output(wait=True)
            # ---- Get the dropdown selection
            model_selection = change["new"]
            # ---- Retrieve the model argument used for `variogram(...)`
            model_name = VARIOGRAM_MODEL_PARAMETER_MAP[model_selection]
            # ---- Get argument names for this particular function
            function_arguments, _ = get_variogram_arguments(model_name)
            # ---- Model arguments (convert to dictionary)
            args_dict = dict(function_arguments)
            # ---- Clear `variogram_settings`
            variogram_settings.clear()
            # ---- Clear existing widgets, if any
            variogram_model_widgets.clear_output()
            # ---- Add the model name to the dictionary
            variogram_settings["model"] = model_name
            # ---- Generate widgets for each parameter
            for param in args_dict.values():
                # ---- Get the parameter name
                parameter = param.name
                # ---- Search reference dictionary for widget parameterization
                if parameter != "distance_lags":
                    # ---- Get default parameterization
                    default_value = get_variogram_defaults(variogram_parameters, parameter)
                    # ---- Get the function argument name
                    arg_name = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["name"]
                    # ---- Get widget-type
                    widget_type = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["widget"]
                    # ---- Get step size
                    step_size = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["step"]
                    # ---- Create the appropriate widget
                    if widget_type == "checkbox":
                        checkbox = widgets.Checkbox(value=default_value, description=arg_name, 
                                                    style={"description_width": "initial"})
                        # ---- Add checkbox entry to `variogram_settings`
                        variogram_settings[parameter] = checkbox
                        # ---- Display the result
                        display(checkbox)
                    elif widget_type == "entry": 
                        entry = widgets.FloatText(value=default_value, description=arg_name, 
                                                style=style, layout=layout, step=step_size)
                        # ---- Add floattext entry to `variogram_settings`
                        variogram_settings[parameter] = entry
                        # ---- Display the result
                        display(entry)            

    # Attach the observer function to the dropdown menu
    dropdown_variogram_model.observe(dynamic_parameters, names="value")

    # Register the updated parameters
    dynamic_parameters({"new": dropdown_variogram_model.value})

    def compute_theoretical_variogram(lags, parameters):
        # Get the current input values
        arg_dict = {
            key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
            for key, value in parameters.items()
        }

        # pass into variogram
        
        distance_lags = lags
        return variogram(distance_lags, arg_dict)
    
    def plot_theoretical_variogram(fig, ax, parameters):
        lags = empirical_params["lags"]
        theoretical_variogram = compute_theoretical_variogram(lags, parameters)
        
        ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
        # plt.plot(lags, theoretical_variogram)
        fig.canvas.draw_idle()

    def on_compute_button_clicked(button):
        # parameters = variogram_widgets
        parameters = variogram_settings
        with plot_output_widget: 
            # out.clear_output(wait=True)  # Clear previous plot
            clear_output(wait=True)
            fig, ax = plot_empirical_variogram(**empirical_params)
            plot_theoretical_variogram(fig, ax, parameters)
            fig.canvas.toolbar_visible = False
            fig.canvas.header_visible = False
            fig.canvas.footer_visible = False
            fig.canvas.resizable = False
            plt.show(fig)

    # Update plot
    compute_theoretical_variogram_button.on_click(on_compute_button_clicked)

    widget_box = widgets.VBox([
        dropdown_variogram_model,
        variogram_model_widgets,
        compute_theoretical_variogram_button
    ])

    return widget_box, variogram_settings

In [ ]:
%matplotlib widget
plot_output_widget = widgets.Output(layout=widgets.Layout(height="550px"))
# empirical_params = {}
loading_label = widgets.Label(value="")
tab_empirical_variogram, general_settings, empirical_params = empirical_variogram_widget()
tab_contents_variogram, variogram_settings = theoretical_variogram_widgets(empirical_params)
# variogram_accordion = widgets.Accordion(layout=dict(width="550px"))
# tab_empirical_variogram = widgets.Label(value="Optimization Parameters tab will go here.")
# tab_contents_variogram = widgets.Label(value="Optimization Parameters tab will go here.")
placeholder_optimization = widgets.Label(value="Optimization Parameters tab will go here.")
tab_contents_optimization = widgets.VBox([placeholder_optimization])

# tab_empirical_variogram, general_settings, empirical_params = empirical_variogram_widget()
full_accordion = stitch_variogram_accordion(tab_empirical_variogram, 
                                            tab_contents_variogram, 
                                            tab_contents_optimization)

widgets.HBox([full_accordion, plot_output_widget])

In [ ]:
variogram_settings

In [ ]:
plot_output_widget = widgets.Output(layout=widgets.Layout(height="550px"))

# dropdown_variogram_model = widgets.Dropdown(
#         options=list(VARIOGRAM_MODEL_PARAMETER_MAP.keys()),
#         value="Bessel-exponential",
#         description="Variogram model",
#         disabled=False,
#         style = {"description_width": "153px"}, 
#         layout = {"width": "403px"}
#     )

# Create text entry widgets (initially empty)
# text_widgets = {
#     param_name: widgets.FloatText(description=param_details['name'], step=param_details['step'])
#     if param_details['widget'] == 'entry' else
#     widgets.Checkbox(description=param_details['name'])
#     for param_name, param_details in THEORETICAL_VARIOGRAM_PARAMETER_MAP.items()
# }

# Create theoretical model computation button
# compute_theoretical_variogram_button = widgets.Button(
#     description='Compute Theoretical Variogram',
#     layout={"width": "405px"},
#     style={'font_weight': 'bold'}
# )

# def get_variogram_defaults(variogram_parameters, argument):
#     DEFAULT_PARAMETERS = {
#         "nugget": 0.0,
#         "sill": 1.0,
#         "hole_effect_range": 0.0,
#         "decay_power": 1.5,
#         "enhance_semivariance": False,
#     }
#     # ---- get variogram config
#     if argument in variogram_parameters.keys():
#         return variogram_parameters[argument]
#     else:
#         return DEFAULT_PARAMETERS[argument]

# # Dynamically populate the model parameter field(s)
# # ---- Initialize the value entry dictionary
# variogram_settings = {}
# # ---- Initialize the Output widget
variogram_model_widgets = widgets.Output()

def dynamic_parameters(change):
    with variogram_model_widgets:
        # ---- Clear previous values
        clear_output(wait=True)
        # ---- Get the dropdown selection
        model_selection = change["new"]
        # ---- Retrieve the model argument used for `variogram(...)`
        model_name = VARIOGRAM_MODEL_PARAMETER_MAP[model_selection]
        # ---- Get argument names for this particular function
        function_arguments, _ = get_variogram_arguments(model_name)
        # ---- Model arguments (convert to dictionary)
        args_dict = dict(function_arguments)
        # ---- Clear `variogram_settings`
        variogram_settings.clear()
        # ---- Clear existing widgets, if any
        variogram_model_widgets.clear_output()
        # ---- Add the model name to the dictionary
        variogram_settings["model"] = model_name
        # ---- Generate widgets for each parameter
        for param in args_dict.values():
            # ---- Get the parameter name
            parameter = param.name
            # ---- Search reference dictionary for widget parameterization
            if parameter != "distance_lags":
                # ---- Get default parameterization
                default_value = get_variogram_defaults(variogram_parameters, parameter)
                # ---- Get the function argument name
                arg_name = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["name"]
                # ---- Get widget-type
                widget_type = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["widget"]
                # ---- Get step size
                step_size = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["step"]
                # ---- Create the appropriate widget
                if widget_type == "checkbox":
                    checkbox = widgets.Checkbox(value=default_value, description=arg_name, 
                                                style={"description_width": "initial"})
                    # ---- Add checkbox entry to `variogram_settings`
                    variogram_settings[parameter] = checkbox
                    # ---- Display the result
                    display(checkbox)
                elif widget_type == "entry": 
                    entry = widgets.FloatText(value=default_value, description=arg_name, 
                                            style=style, layout=layout, step=step_size)
                    # ---- Add floattext entry to `variogram_settings`
                    variogram_settings[parameter] = entry
                    # ---- Display the result
                    display(entry)            

# Attach the observer function to the dropdown menu
dropdown_variogram_model.observe(dynamic_parameters, names="value")

# Register the updated parameters
dynamic_parameters({"new": dropdown_variogram_model.value})

def plot_theoretical_variogram(fig, ax, empirical_params, parameters):
    lags = empirical_params["lags"]
    theoretical_variogram = compute_theoretical_variogram(parameters, lags)
    
    ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
    # plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()

def on_compute_button_clicked(button):
    # parameters = variogram_widgets
    parameters = variogram_settings
    with plot_output_widget: 
        # out.clear_output(wait=True)  # Clear previous plot
        clear_output(wait=True)
        fig, ax = plot_empirical_variogram(**empirical_params)
        plot_theoretical_variogram(fig, ax, empirical_params, parameters)
        fig.canvas.toolbar_visible = False
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        fig.canvas.resizable = False
        plt.show(fig)
        
compute_theoretical_variogram_button.on_click(on_compute_button_clicked)

widget_box = widgets.VBox([
    dropdown_variogram_model,
    variogram_model_widgets,
    compute_theoretical_variogram_button
])

def compute_theoretical_variogram(variogram_settings, lags):
    # Get the current input values
    arg_dict = {
        key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
        for key, value in variogram_settings.items()
    }

    # pass into variogram
    
    distance_lags = lags
    return variogram(distance_lags, arg_dict)

with plot_output_widget:
    fig, ax = plot_empirical_variogram(**empirical_params)
    plt.show(fig)


widgets.HBox([widget_box, plot_output_widget])

In [ ]:
def plot_theoretical_variogram(fig, ax, empirical_params, parameters):
    lags = empirical_params["lags"]
    theoretical_variogram = compute_theoretical_variogram(parameters, lags)
    
    ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
    # plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()

In [ ]:
# Create general settings value entry widgets
# ---- Number of lags
n_lags_entry = widgets.IntText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["n_lags"],
                                description="Number of lags",
                                step=1, style=style, layout=layout)
# ---- Lag (increment) resolution
lag_resolution_entry =  widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"], 
                                            description='Lag resolution', 
                                            step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"]/2, 
                                            style=style, layout=layout)
# ---- Azimuth range 
azimuth_range_entry = widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"], 
                                        description='Azimuth Range', 
                                        step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"]/360, 
                                        style=style, layout=layout)

# Add variable dropdown widget
empirical_variogram_dropdown = widgets.Dropdown(
    options=list(["Biomass (biomass density)", "Abundance (number density)"]),
    value="Biomass (biomass density)",
    description="Transect variable",
    disabled=False,
    style = {"description_width": "150px"}, 
    layout = {"width": "400px"}
)  

# Add empirical variogram button
compute_empirical_variogram_button = widgets.Button(description="Compute Empirical Variogram",
                                                    layout={"width": "405px"},
                                                    style={"font_weight": "bold"})    

settings_dict.update({"variable": "biomass_density"})

def update_settings_dict(change):
    settings_dict.update({
        "variable": (
            "biomass_density" if empirical_variogram_dropdown.value == "Biomass (biomass density)" 
            else "number_density"
        ),
    })

# Initialize the value dictionary
general_settings = {
    "n_lags": n_lags_entry.value,
    "lag_resolution": lag_resolution_entry.value,
    "azimuth_range": azimuth_range_entry.value,
    "variable": empirical_variogram_dropdown.value,
}

def compute_empirical_variogram(transect_data, general_settings, settings_dict):
    # Attach the update function to the dropdown value change event
    empirical_variogram_dropdown.observe(update_settings_dict, names='value')
    # Compute the distance lags
    distance_lags = np.arange(1, general_settings["n_lags"]) * general_settings["lag_resolution"]
    # ---- Compute the maximum range
    max_range = general_settings["n_lags"] * general_settings["lag_resolution"]

    # Update the general settings for computing the empirical variogram
    general_settings.update({"force_lag_zero": True, "distance_lags": distance_lags, 
                            "range": max_range})
    
    # Compute the empirical variogram
    lags, gamma_h, lag_counts, _ = empirical_variogram(
        transect_data, general_settings, settings_dict
    )

    # Return the outputs
    return lags, gamma_h, lag_counts, general_settings

empirical_params = {}
# # Button-click
def click_empirical_variogram_button(button, empirical_params):
    # Show the loading indicator
    loading_label.value = "Computing the empirical variogram, please wait..."
    # Compute the empirical variogram
    
    lags, gamma_h, lag_counts, _ = compute_empirical_variogram(transect_data, general_settings, settings_dict)
    # empirical_params = {}
    empirical_params["lags"] = lags
    empirical_params["gamma_h"] = gamma_h
    empirical_params["lag_counts"] = lag_counts
    # Clear the previous plot
    plot_output_widget.clear_output()

    # Plot the new empirical variogram
    with plot_output_widget:
        fig, ax = plot_empirical_variogram(lags, gamma_h, lag_counts)
        plt.show(fig)

    # Hide the loading indicator
    loading_label.value = ""

    # return empirical_params
    # return lags, gamma_h, lag_counts

# Attach the button click event to the handler function
# empirical_parmas = compute_empirical_variogram_button.on_click(click_empirical_variogram_button)

compute_empirical_variogram_button.on_click(lambda button: click_empirical_variogram_button(button, empirical_params))

# compute_empirical_variogram_button.on_click(click_empirical_variogram_button)

# Create helper function that updates the current entry values
def update_general_settings(change):
    # ---- Number of lags
    general_settings["n_lags"] = n_lags_entry.value
    # ---- Lag (increment) resolution
    general_settings["lag_resolution"] = lag_resolution_entry.value
    # ---- Azimuth range 
    general_settings["azimuth_range"] = azimuth_range_entry.value

def register_observers(widgets, handler, names='value'):
    for widget in widgets:
        widget.observe(handler, names=names)

# Register the widget observers
register_observers([n_lags_entry, lag_resolution_entry, azimuth_range_entry], 
                    update_general_settings)

# Vertically concatenate/bundle widgets
widget_box = widgets.VBox(
    [n_lags_entry, lag_resolution_entry, azimuth_range_entry, empirical_variogram_dropdown,
    compute_empirical_variogram_button]
)

# Return the widget and settings dictionary
widget_box

In [ ]:
empirical_params

In [ ]:


style = {'description_width': '150px'}
layout = {'width': '400px'}
def get_variogram_defaults(variogram_parameters, argument):
    DEFAULT_PARAMETERS = {
        "nugget": 0.0,
        "sill": 1.0,
        "hole_effect_range": 0.0,
        "decay_power": 1.5,
        "enhance_semivariance": False,
    }
    # ---- get variogram config
    for argument in DEFAULT_PARAMETERS.keys():
        if argument in variogram_parameters.keys():
            return variogram_parameters[argument]
        else:
            return DEFAULT_PARAMETERS[argument]


# Define your dropdown widget
dropdown_variogram_model = widgets.Dropdown(
    options=list(VARIOGRAM_MODEL_PARAMETER_MAP.keys()),
    value="Bessel-exponential",
    description="Variogram model",
    disabled=False,
    style = {"description_width": "153px"}, 
    layout = {"width": "403px"}
)

# Create text entry widgets (initially empty)
text_widgets = {
    param_name: widgets.FloatText(description=param_details['name'], step=param_details['step'])
    if param_details['widget'] == 'entry' else
    widgets.Checkbox(description=param_details['name'])
    for param_name, param_details in THEORETICAL_VARIOGRAM_PARAMETER_MAP.items()
}

def get_variogram_defaults(variogram_parameters, argument):
    DEFAULT_PARAMETERS = {
        "nugget": 0.0,
        "sill": 1.0,
        "hole_effect_range": 0.0,
        "decay_power": 1.5,
        "enhance_semivariance": False,
    }
    # ---- get variogram config
    if argument in variogram_parameters.keys():
        return variogram_parameters[argument]
    else:
        return DEFAULT_PARAMETERS[argument]

# Dynamically populate the model parameter field(s)
# ---- Initialize the value entry dictionary
variogram_settings = {}
# ---- Initialize the Output widget
variogram_model_widgets = widgets.Output()

def dynamic_parameters(change):
    with variogram_model_widgets:
        # ---- Clear previous values
        clear_output(wait=True)
        # ---- Get the dropdown selection
        model_selection = change["new"]
        # ---- Retrieve the model argument used for `variogram(...)`
        model_name = VARIOGRAM_MODEL_PARAMETER_MAP[model_selection]
        # ---- Get argument names for this particular function
        function_arguments, _ = get_variogram_arguments(model_name)
        # ---- Model arguments (convert to dictionary)
        args_dict = dict(function_arguments)
        # ---- Clear `variogram_settings`
        variogram_settings.clear()
        # ---- Clear existing widgets, if any
        variogram_model_widgets.clear_output()
        # ---- Add the model name to the dictionary
        variogram_settings["model"] = model_name
        # ---- Generate widgets for each parameter
        for param in args_dict.values():
            # ---- Get the parameter name
            parameter = param.name
            # ---- Search reference dictionary for widget parameterization
            if parameter != "distance_lags":
                # ---- Get default parameterization
                default_value = get_variogram_defaults(variogram_parameters, parameter)
                # ---- Get the function argument name
                arg_name = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["name"]
                # ---- Get widget-type
                widget_type = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["widget"]
                # ---- Get step size
                step_size = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["step"]
                # ---- Create the appropriate widget
                if widget_type == "checkbox":
                    checkbox = widgets.Checkbox(value=default_value, description=arg_name, 
                                                style={"description_width": "initial"})
                    # ---- Add checkbox entry to `variogram_settings`
                    variogram_settings[parameter] = checkbox
                    # ---- Display the result
                    display(checkbox)
                elif widget_type == "entry": 
                    entry = widgets.FloatText(value=default_value, description=arg_name, 
                                            style=style, layout=layout, step=step_size)
                    # ---- Add floattext entry to `variogram_settings`
                    variogram_settings[parameter] = entry
                    # ---- Display the result
                    display(entry)            

# Attach the observer function to the dropdown menu
dropdown_variogram_model.observe(dynamic_parameters, names="value")

# Register the updated parameters
dynamic_parameters({"new": dropdown_variogram_model.value})

widget_box = widgets.VBox([
    dropdown_variogram_model,
    variogram_model_widgets,
])

widget_box